In [2]:
import pandas as pd

In [3]:
datasetsentences = pd.read_csv('data/01_raw/datasetSentences.txt', sep='\t')
datasetsplit = pd.read_csv('data/01_raw/datasetSplit.txt', sep=',')

datasetsentences.head()

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,"Emerges as something rare , an issue movie tha..."


In [5]:
sentences = {}

#fix some typo from sentences
for index, row in datasetsentences.iterrows():
    row['sentence'] = row['sentence'].replace('-LRB-','(')
    row['sentence'] = row['sentence'].replace('-RRB-',')')
    row['sentence'] = row['sentence'].replace('Â', '')
    row['sentence'] = row['sentence'].replace('Ã©', 'e')
    row['sentence'] = row['sentence'].replace('Ã¨', 'e')
    row['sentence'] = row['sentence'].replace('Ã¯', 'i')
    row['sentence'] = row['sentence'].replace('Ã³', 'o')
    row['sentence'] = row['sentence'].replace('Ã´', 'o')
    row['sentence'] = row['sentence'].replace('Ã¶', 'o')
    row['sentence'] = row['sentence'].replace('Ã±', 'n')
    row['sentence'] = row['sentence'].replace('Ã¡', 'a')
    row['sentence'] = row['sentence'].replace('Ã¢', 'a')
    row['sentence'] = row['sentence'].replace('Ã£', 'a')
    row['sentence'] = row['sentence'].replace('\xc3\x83\xc2\xa0', 'a')
    row['sentence'] = row['sentence'].replace('Ã¼', 'u')
    row['sentence'] = row['sentence'].replace('Ã»', 'u')
    row['sentence'] = row['sentence'].replace('Ã§', 'c')
    row['sentence'] = row['sentence'].replace('Ã¦', 'ae')
    row['sentence'] = row['sentence'].replace('Ã­', 'i')
    row['sentence'] = row['sentence'].replace('\xa0', ' ')
    row['sentence'] = row['sentence'].replace('\xc2', '')
    sentences[row['sentence_index']] = row['sentence']

In [6]:
datasetsplit.head()

,sentence_index,splitset_label
0,1,1
1,2,1
2,3,2
3,4,2
4,5,2


In [7]:
train={}
test={}
dev={}
sents=[]
for index, row in datasetsplit.iterrows():
    if row['splitset_label'] == 1:
        train[sentences[row['sentence_index']]] = 0
        sents.append(sentences[row['sentence_index']])
    elif row['splitset_label'] ==2:
        test[sentences[row['sentence_index']]] = 0
    elif row['splitset_label'] ==3:
        dev[sentences[row['sentence_index']]] = 0

In [8]:
dictionary = pd.read_csv('data/01_raw/dictionary.txt', sep='|', header = None, names=['phrase', 'phraseId'])
sentiment_labels = pd.read_csv('data/01_raw/sentiment_labels.txt', sep='|')

In [9]:
dictionary.head()

,phrase,phraseId
0,!,0
1,! ',22935
2,! '',18235
3,! Alas,179257
4,! Brilliant,22936


In [10]:
train_sent = train.copy()
string = ' '.join(sents)

#fix some typo from phrase
for index, row in dictionary.iterrows():
    row['phrase'] = row['phrase'].replace('é','e')
    row['phrase'] = row['phrase'].replace('è','e')
    row['phrase'] = row['phrase'].replace('ï','i')
    row['phrase'] = row['phrase'].replace('í','i')
    row['phrase'] = row['phrase'].replace('ó','o')
    row['phrase'] = row['phrase'].replace('ô','o')
    row['phrase'] = row['phrase'].replace('ö','o')
    row['phrase'] = row['phrase'].replace('á','a')
    row['phrase'] = row['phrase'].replace('â','a')
    row['phrase'] = row['phrase'].replace('ã','a')
    row['phrase'] = row['phrase'].replace('à','a')
    row['phrase'] = row['phrase'].replace('ü','u')
    row['phrase'] = row['phrase'].replace('û','u')
    row['phrase'] = row['phrase'].replace('ñ','n')
    row['phrase'] = row['phrase'].replace('ç','c')
    row['phrase'] = row['phrase'].replace('æ','ae')
    row['phrase'] = row['phrase'].replace('\xa0', ' ')
    row['phrase'] = row['phrase'].replace('\xc2', '')
    if row['phrase'] in string:
        train[row['phrase']] = row['phraseId']
    if row['phrase'] in test:
        test[row['phrase']] = row['phraseId']
    if row['phrase'] in train_sent:
        train_sent[row['phrase']] = row['phraseId']
    if row['phrase'] in dev:
        dev[row['phrase']] = row['phraseId']

In [11]:
sentiment_labels.head()

,phrase ids,sentiment values
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708


In [12]:
labels = {}
for index, row in sentiment_labels.iterrows():
    labels[row['phrase ids']] = float(row['sentiment values'])

In [13]:
for key in train:
    train[key]=labels[train[key]]
for key in train_sent:
    train_sent[key]=labels[train_sent[key]]
for key in test:
    test[key]=labels[test[key]]
for key in dev:
    dev[key]=labels[dev[key]]

In [14]:
print(len(train))
print(len(train_sent))
print(len(test))
print(len(dev))


170132
8534
2210
1100


In [15]:
train_phrase = pd.DataFrame(list(train.items()), columns=['phrase', 'sentiment'])
train_sentence = pd.DataFrame(list(train_sent.items()), columns=['sentence', 'sentiment'])
test_set = pd.DataFrame(list(test.items()), columns=['sentence', 'sentiment'])
dev_set = pd.DataFrame(list(dev.items()), columns=['sentence', 'sentiment'])

In [16]:
train_phrase.to_csv('data/02_processed/train_phrase.csv', index = False)
train_sentence.to_csv('data/02_processed/train_sentence.csv', index = False)
test_set.to_csv('data/02_processed/test.csv', index = False)
dev_set.to_csv('data/02_processed/dev.csv', index = False)

In [17]:
#transform sentiment value into category
def mapping_sentiment(val):
    """
    number -> number
    convert sentiment value into categorical value using the following cut-offs:
    [0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.6, 0.8], (0.8, 1.0]
    """
    if val >= 0 and val <= 0.2:
        return 1
    elif val > 0.2 and val <= 0.4:
        return 2
    elif val > 0.4 and val <= 0.6:
        return 3
    elif val > 0.6 and val <= 0.8:
        return 4
    elif val > 0.8 and val <= 1:
        return 5

In [18]:
train_phrase['sentiment'] = train_phrase['sentiment'].apply(lambda x: mapping_sentiment(x))
train_sentence['sentiment'] = train_sentence['sentiment'].apply(lambda x: mapping_sentiment(x))
test_set['sentiment'] = test_set['sentiment'].apply(lambda x: mapping_sentiment(x))
dev_set['sentiment'] = dev_set['sentiment'].apply(lambda x: mapping_sentiment(x))

In [19]:
train_phrase.to_csv('data/02_processed/cat_train_phrase.csv', index = False)
train_sentence.to_csv('data/02_processed/cat_train_sentence.csv', index = False)
test_set.to_csv('data/02_processed/cat_test.csv', index = False)
dev_set.to_csv('data/02_processed/cat_dev.csv', index = False)